If you're running this jupyter notebook in a Google Colab please please uncomment lines and run this cell.
Otherwise ignore it since the files will be available to you locally

In [ ]:
# !wget -P data/ https://raw.githubusercontent.com/paramm-team/data_processing/main/src/input/data/Digatron.csv
# !wget -P data/ https://raw.githubusercontent.com/paramm-team/data_processing/main/src/input/data/Gamry.DTA
# !wget -P data/ https://raw.githubusercontent.com/paramm-team/data_processing/main/src/input/data/Maccor.csv
# !wget -P data/ https://raw.githubusercontent.com/paramm-team/data_processing/main/src/input/data/Novonix.csv

When you execute this command in a Jupyter Notebook, pip will clone the data_processing repository from GitHub and install it into your Python environment. This method of installation is often used for packages that are in development or when you want to install a specific version of a package that is not available through PyPI.

In [ ]:
%pip install git+https://github.com/paramm-team/data_processing.git

The line import src in a Jupyter Notebook is a Python statement that imports a module named src into the current namespace, allowing you to use its functions, classes, and variables within your notebook.

In [1]:
import pbdp
from pbdp import segment
# This creates a logger object that can be used to log messages in the default pbdp way
_logger = pbdp.create_logger()

The result of this function call will be a string that represents the path to the data directory. This path will be platform-independent, meaning it will use the correct path separators for Unix (/) or Windows (\) if run locally or on Google Colab.

Files available in this folder for testing purposes are: Digatron.csv, Maccor.csv, Novonix.csv, Gamry.DTA

In [2]:
from pathlib import Path
import platform
if 'google.colab' in str(get_ipython()):
    path = Path('data/')
else:
    path = Path(pbdp.__path__[0], "input", "data") 
    print(path)

/Users/pipgrylls/Code/data_processing/pbdp/input/data


Great! Up until this point the notebook is identical with the Pre-processing one you just saw potentially, if not welcome! Let's **pre-process one file here so we can use it later**. For reference look at the **Data Importer Pre-processing Example** if you have any questions about this step.

In [3]:
parser = pbdp.Parser()
data = parser.data_importer(path_or_file=path / "Digatron.csv", state_option="yes")

There are several ways you can segment your battery data based on your needs. As an overview you can slice it by the following requests **Step, Time, Current, Voltage, Power, Constant Current and Voltage, Pulses and a chain of actions**. Finally you can choose to **reset the Time column (reset=True)** for them all Let's explore each approach below!

Step Segmentation (acts as list slicing)



In [ ]:
segment.segment_data(data=data, requests=["step"]) # returns the whole dataset

In [ ]:
segment.segment_data(data=data, requests=["step 10:20"]) # returns slice of data for steps between 10 and 20

In [ ]:
segment.segment_data(data=data, requests=["step 5:5"]) # returns just the step 5

Time Segmentation (acts as list slicing)

In [ ]:
segment.segment_data(data=data, requests=["time"]) # returns the whole dataset

In [ ]:
segment.segment_data(data=data, requests=["time 50/200"]) # returns slice of data for time between 50 and 200 seconds

In [ ]:
segment.segment_data(data=data, requests=["time 50/50"]) # returns just the 50 seconds point

Current Segmentation (Rest, Charge, Discharge)

In [ ]:
segment.segment_data(data=data, requests=["rest"]) # returns all the segments for rest periods

In [ ]:
segment.segment_data(data=data, requests=["charging"]) # returns all the segments for charging periods

In [6]:
segment.segment_data(data=data, requests=["charging 0.5A"], reset=True) # returns the segments for charging periods at 0.5A

In [ ]:
segment.segment_data(data=data, requests=["dischg"]) # returns all the segments for discharging periods

In [ ]:
segment.segment_data(data=data, requests=["dischg 0.5A"]) # returns the segments for discharging periods at -0.5A

Constant Current Segmentation

In [ ]:
segment.segment_data(data=data, requests=["cc"]) # returns all the segments for Constant Current

In [ ]:
segment.segment_data(data=data, requests=["cc 1.67A"]) # returns the segments for Constant Current at 1.67A

Constant Voltage Segmentation

In [ ]:
segment.segment_data(data=data, requests=["cv"]) # returns all the segments for Constant Voltage

In [ ]:
segment.segment_data(data=data, requests=["cv 4.2V"]) # returns the segments for Constant Voltage at 4.2V

Constant Power Segmentation

In [ ]:
segment.segment_data(data=data, requests=["power"]) # returns all the segments for Constant Voltage

In [ ]:
segment.segment_data(data=data, requests=["power 1.05W"]) # returns the segments for Constant Power at 1.05W

CCCV (Constant Current Constant Voltage) Segmentation

In [ ]:
segment.segment_data(data=data, requests=["cccv"]) # returns all the segments for Constant Current Constant Voltage

In [ ]:
# returns the segments for Constant Current Constant Voltage at 1.67A and 4.2V (first have the amps and then volts values)
segment.segment_data(data=data, requests=["cccv 1.67A 4.2V"])

In [ ]:
# returns the segments for Constant Current Constant Voltage strictly at 4.2V and any Constant Current (passing 1.67A will return CCCV strictly with 1.67A and any constant voltage)
segment.segment_data(data=data, requests=["cccv 4.2V"])

Pulse Segmentation

In [5]:
segment.segment_data(data=data, requests=["pulse"], reset=True) # returns all the segments for pulses

[      Step Number Status Step Time  Time [s] Cycle Cycle Level  \
 3602          2.0    DCH     0.000     0.000     0           0   
 3603          2.0    DCH     1.002     1.002     0           0   
 3604          2.0    DCH     1.968     1.968     0           0   
 3605          2.0    DCH     3.053     3.053     0           0   
 3606          2.0    DCH     4.152     4.152     0           0   
 ...           ...    ...       ...       ...   ...         ...   
 3762          2.0    DCH   160.074   160.074     0           0   
 3763          2.0    DCH   161.025   161.025     0           0   
 3764          2.0    DCH   162.054   162.054     0           0   
 3765          2.0    DCH   163.013   163.013     0           0   
 3766          2.0    DCH   163.284   163.284     0           0   
 
                  Procedure  Voltage [V]  Current [A]  AmpHrs [Ah]   AhPrev  \
 3602  LTA_LG50_RPT_Ext_All      3.04213     -1.66895      0.00000  0.00000   
 3603  LTA_LG50_RPT_Ext_All      3.0

In [ ]:
segment.segment_data(data=data, requests=["pulse -1.67A"]) # returns the segments for pulses at -1.67A

**More complex queries** (2 different actions proceeding each other)

There are many ways you can alternate and construct your query hence we'll explore just a few below. Available options to build with: **rest, charging, discharging, cc, cv, cccv, pulse**. Please add **A, V or amp** (if the two options are both regarging current)

As a trick if you require a **rest period before or after a certain action** this is how you'd use our tool for that.

In [ ]:
segment.segment_data(data=data, requests=["cv, rest"]) # returns the segments for cv plus the rest period following

In [ ]:
segment.segment_data(data=data, requests=["rest, cc 1.67A"]) # returns the segments for cc plus the rest period proceeding

**Resetting the Time** column of a segment or list of segments

In [ ]:
pulse = segment.segment_data(data=data, requests=["pulse -10A"])
pulse = segment.reset_time(data=pulse) # resetting the time of the pulses

**Add Rest Period** to any segment before and after depending of availability

In [ ]:
pulse = segment.segment_data(data=data, requests=["pulse -10A"])
pulse = segment.find_rest(data=data, segments=pulse) # adding the rest periods before and after if they exist